<a href="https://colab.research.google.com/github/hissain/ml/blob/main/advance/TuneSmallLLMinRHLFloop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

                   ┌────────────────────────────┐
                   │   Pretrained Base Model     │
                   │  (e.g., TinyLlama/Qwen/etc) │
                   └─────────────┬──────────────┘
                                 │
                                 ▼
                    ┌────────────────────────┐
                    │ 1. Supervised Fine-Tuning│
                    │ (SFT on instructions)    │
                    └─────────────┬────────────┘
                                  │
                     SFT Model    │
                                  ▼
          ┌───────────────────────────────────────────┐
          │ 2. Preference Data Collection (Human or   │
          │    Simulated)                             │
          │   • Generate two (or more) responses      │
          │   • Rank: “chosen” > “rejected”           │
          └─────────────────────┬─────────────────────┘
                                │
                                ▼
                    ┌─────────────────────────┐
                    │ 3. Reward Model Training │
                    │  (learns ranking signal) │
                    └──────────────┬───────────┘
                                   │
                                   ▼
               ┌───────────────────────────────────┐
               │ 4. RLHF Loop (e.g., PPO Training) │
               │   • SFT model becomes the “policy”│
               │   • Generate outputs              │
               │   • Reward model scores them      │
               │   • PPO updates the policy        │
               └───────────────────┬───────────────┘
                                   │
                                   ▼
                   ┌─────────────────────────────┐
                   │       RLHF-Trained Model     │
                   │ (aligned, safer, higher-quality) │
                   └─────────────────────────────┘


# 🎯 Explanation of Flow
## 1. Pretrained Base Model

A small LLM (0.5B–3B parameters) serves as the starting point.

## 2. SFT (Supervised Fine-Tuning)

The model is trained on human-written or synthetic instruction/response pairs.
This makes the model follow instructions before RLHF.

## 3. Preference Data Collection

For each prompt:

Generate multiple candidate responses

A human (or synthetic reward function) ranks them

“Chosen” > “Rejected”

This is the core dataset for RLHF.

## 4. Reward Model Training

A reward model learns to predict the ranking.
It turns preferences into a scalar score.

## 5. PPO (or another RL method)

The SFT model becomes the policy model.

**Process**


1. Generate responses from the policy

2. Reward model scores them

3. PPO updates weights toward high-reward behavior

4. KL penalty prevents drifting too far from SFT behavior

**Output: RLHF Final Model**

 Safer

 More helpful

More human-aligned

Higher-quality responses

# Proximal Policy Optimization (PPO)

**PPO** is a reinforcement learning algorithm used in RLHF to fine-tune language models.
Its goal is to update a model’s behavior toward preferred outputs **without letting the
model change too much at once**, which prevents instability or collapse.

---

## 🔶 Why PPO?

Large language models are sensitive. If updates are too big, the model may:
- forget language ability  
- generate unstable text  
- diverge during training  

PPO limits the update size using a **clipped objective**, ensuring training remains stable.

---

## 🔶 Core Idea

Compute the probability ratio between the new and old policy:



# Install Dependencies

In [1]:
!pip install -q transformers accelerate datasets trl peft bitsandbytes sentencepiece wandb
!pip install -q einops


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 15.9 MB/s eta 0:00:00


# Import

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset, load_dataset
from trl import SFTTrainer, PPOTrainer, AutoModelForCausalLMWithValueHead
from peft import LoraConfig, get_peft_model
import wandb


# Configuration

In [ ]:
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
DEVICE = "cuda"

USE_WANDB = False
WANDB_PROJECT = "rlhf-simulated-feedback"

if USE_WANDB:
    wandb.init(project=WANDB_PROJECT)


# Load Tokenizer & Model (8-bit for Colab)

In [ ]:
!pip install -U bitsandbytes

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

# Loading without 8-bit quantization to bypass persistent ImportError
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
).eval()

# Create Synthetic Training Data

In [ ]:
synthetic_data = {
    "prompt": [
        "Explain reinforcement learning in one paragraph.",
        "Write a short poem about rockets.",
        "Describe why fine-tuning is useful.",
        "Explain the difference between PPO and DQN."
    ] * 250,

    "response": [
        "Reinforcement learning is a computational framework where agents learn optimal behaviors through trial and error...",
        "Rockets roar into the sky, carrying dreams of distant stars...",
        "Fine-tuning allows pretrained models to specialize...",
        "PPO is policy-gradient based while DQN is value-based..."
    ] * 250
}

ds_sft = Dataset.from_dict(synthetic_data)
ds_sft

# Prepare LoRA for Efficient Training

In [ ]:
lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)


# SFT Training

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer # Added AutoTokenizer
from trl import SFTTrainer, SFTConfig # Import SFTConfig
from peft import LoraConfig, get_peft_model

MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0" # Added MODEL_NAME definition

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

# Loading without 8-bit quantization to bypass persistent ImportError
model_sft = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
)
model_sft = get_peft_model(model_sft, lora_config)

# Replace TrainingArguments with SFTConfig
sft_config = SFTConfig(
    output_dir="./sft-output",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    logging_steps=50,
    learning_rate=2e-4,
    # Set completion_only_loss to False to train on the full prompt+response
    completion_only_loss=False,
    bf16=False, # Explicitly disable bf16
    fp16=True,  # Explicitly enable fp16 if available
)

def formatting_func(example):
    # Combine prompt and response into a single string as expected by SFTTrainer for full text training
    return f"Prompt: {example['prompt']}\nResponse: {example['response']}"

# Add 'completion' field to ds_sft to satisfy SFTTrainer's internal tokenizer
def add_completion_field(example):
    example["completion"] = example["response"]
    return example

ds_sft_processed = ds_sft.map(add_completion_field)

sft_trainer = SFTTrainer(
    model=model_sft,
    train_dataset=ds_sft_processed, # Use the processed dataset
    formatting_func=formatting_func, # Use the updated formatting_func
    args=sft_config, # Pass SFTConfig object here
)

sft_trainer.train()
model_sft.save_pretrained("./sft-trained")

# Generate “Better” and “Worse” Responses (Simulated Human Feedback)

In [ ]:
import random
from transformers import AutoTokenizer # Added AutoTokenizer

MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0" # Added MODEL_NAME definition
DEVICE = "cuda" # Added DEVICE definition

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

def generate_variants(prompt):
    # low-effort response
    short = model_sft.generate(
        **tokenizer(prompt, return_tensors="pt").to(DEVICE),
        max_new_tokens=32
    )
    # detailed response
    long = model_sft.generate(
        **tokenizer(prompt, return_tensors="pt").to(DEVICE),
        max_new_tokens=128
    )
    return (
        tokenizer.decode(short[0], skip_special_tokens=True),
        tokenizer.decode(long[0], skip_special_tokens=True)
    )

pairs = []
for p in synthetic_data["prompt"][:600]:
    bad, good = generate_variants(p)
    pairs.append({"prompt": p, "chosen": good, "rejected": bad})

ds_preferences = Dataset.from_list(pairs)
ds_preferences

# Train a Reward Model

In [ ]:
from transformers import AutoModelForSequenceClassification

reward_model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=1,
    device_map="auto"
)

def preprocess_reward(batch):
    text = [f"Prompt: {p}\nResponse: {c}" for p, c in zip(batch["prompt"], batch["chosen"])]
    text_bad = [f"Prompt: {p}\nResponse: {r}" for p, r in zip(batch["prompt"], batch["rejected"])]
    batch["text_chosen"] = text
    batch["text_rejected"] = text_bad
    return batch

displayed_ds_preferences = ds_preferences # Ensure ds_preferences is defined and accessible here
ds_reward = displayed_ds_preferences.map(preprocess_reward)

from trl import RewardTrainer

reward_trainer = RewardTrainer(
    model=reward_model,
    tokenizer=tokenizer,
    train_dataset=ds_reward,
    args=dict(
        per_device_train_batch_size=4,
        num_train_epochs=1,
        output_dir="./reward-model"
    )
)

reward_trainer.train()
reward_model.save_pretrained("./reward-model")

# RLHF PPO Training

In [ ]:
from transformers import AutoModelForCausalLM
from trl import AutoModelForCausalLMWithValueHead, PPOTrainer

# Load the base SFT-trained model first
base_ppo_model = AutoModelForCausalLM.from_pretrained(
    "./sft-trained",
    device_map="auto"
)

# Then wrap it with AutoModelForCausalLMWithValueHead
ppo_model = AutoModelForCausalLMWithValueHead.from_pretrained(
    base_ppo_model # Pass the loaded base model
)

ppo_trainer = PPOTrainer(
    model=ppo_model,
    tokenizer=tokenizer,
    reward_model=reward_model,
    dataset=ds_sft,
    args=dict(
        learning_rate=1e-5,
        batch_size=4,
        mini_batch_size=2,
        target_kl=1.0,
        output_dir="./ppo-rlhf"
    )
)

generation_kwargs = dict(
    max_new_tokens=100,
    pad_token_id=tokenizer.eos_token_id
)

ppo_trainer.train(generation_kwargs=generation_kwargs)
ppo_model.save_pretrained("./ppo-final")

# Test the Final Model

In [ ]:
prompt = "Explain why PPO works well in reinforcement learning."
inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)

output = ppo_model.generate(
    **inputs,
    max_new_tokens=150,
    pad_token_id=tokenizer.eos_token_id
)

print(tokenizer.decode(output[0], skip_special_tokens=True))
